# Chapter 1. Basics

## Python numerical types

### Decimal type

For applications that require decimal digits with accurate arithmetic operations, use the `Decimal` type from the `decimal` module in the Python Standard Library:

In [1]:
from decimal import Decimal
num1 = Decimal('1.1')
num2 = Decimal('1.563')
assert Decimal('2.663') == num1 + num2

Certain numbers such as `0.1` cannot be represented exactly using a finite sum of powers of 2. `0.1` has a binary expansion `0.000110011...`, which does not terminate. Any floating-point representation of this number will therefore carry a small error.

In [2]:
assert 2.663 != 1.1 + 1.563

The `decimal` package also provides a `Context` object, which allows fine-grained control over the precision, display, and attributes of `Decimal` objects:

In [3]:
assert Decimal('1.4641') == num1**4

from decimal import localcontext
with localcontext() as ctx:
    ctx.prec = 3
    assert Decimal('1.46') == num1**4

> When we set the precision to `3`, rather than the default `28`, we see that the fourth power of 1.1 is rounded to three significant figures.

This means that context can be freely modified inside the `with` block, and will be returned to the default at the end.

### Fraction type

The `Fraction` type from the `fractions` module in the Python Standard Library, simply stores two integers (the numerator and the denominator), and arithmetic is performed using the basic rule for arithmetic of fractions.

In [4]:
from fractions import Fraction
fr1 = Fraction(1, 3)
fr2 = Fraction(1, 7)
assert Fraction(10, 21) == fr1 + fr2
assert Fraction(4, 21) == fr1 - fr2
assert Fraction(1, 21) == fr1 * fr2
assert Fraction(7, 3) == fr1 / fr2

## Basic mathematical functions

The `math` module in the Python Standard Library provides all of the standard mathematical functions, along with common constants and some utility functions:

In [5]:
import math
assert 2.0 == math.sqrt(4)
assert math.isclose(math.sin(math.pi/4), math.cos(math.pi/4))
assert math.isclose(1.0, math.tan(math.pi/4))

The `log` function in the `math` module performs logarithms.

$$
\begin{aligned}
y = \log_b{x} &\iff x = b^y\\
y = \ln{(x)} &\iff x = e^y
\end{aligned}
$$

Where the constant $e$ is base of the natural logarithm, sometimes known as [Napier's constant](https://mathworld.wolfram.com/e.html). The constant can be accessed using `math.e`.

In [6]:
assert math.log(10) == math.log(10, math.e)
assert math.log(100, 10) == 2.0

In addition, the `math` contains various number of theoretic and combinatorial functions.

The `comb(n, k)` returns the number of ways to choose `k` items from a collection of `n` without repeats if order is not important. This number is sometimes written $\displaystyle {^n}C_k = \binom{n}{k} = \frac{n!}{k!(n-k)!}$.

In [7]:
assert 10 == math.comb(5, 2)

The `factorial(n)` returns the factorial
$n! = n(n-1)(n-2)\cdots 1$.

In [8]:
assert 120 == math.factorial(5)

There are also a number of functions for working with floating-point numbers. As `0.1` carries a small error, Python built-in function `sum` cannot return an accurate sum of values in an iterable, while `math.fsum` avoids loss of precision by tracking multiple intermediate partial sums.

In [9]:
nums = [0.1]*10    # a list containing 0.1 ten items
assert math.isclose(1.0, sum(nums))
assert 1.0 == math.fsum(nums)

The `math` module is a good choice if you need to apply a function to a relatively small collection of numbers. If you want to these functions to a large collection of data simultaneously, it is better to use their equivalents from the NumPy package, which are more efficient for working with arrays.

## NumPy arrays

In [10]:
import numpy as np

The basic type provided by the NumPy library is the `ndarray` type (henceforth referred to as a NumPy array).

- The NumPy array type is a Python wrapper around an underlying C array structure. The array operations are implemented in C and optimized for performance.
- NumPy arrays must consist of homogeneous data (could be a pointer to an arbitrary Python object).
- Under the hood, a NumPy array of any shape is a buffer containing the raw data as a flat (one-dimensional) array, and a collection of additional metadata that specifies details such as the type of the elements.

NumPy provides a number of [universal functions](https://numpy.org/doc/stable/user/basics.ufuncs.html).

- A universal function (or ufunc for short) is a function that operates on ndarrays in an element-by-element fashion, supporting array broadcasting, type casting, and several other standard features.
- That is, a ufunc is a “vectorized” wrapper for a function that takes a fixed number of specific inputs and produces a fixed number of specific outputs.

*Broadcasting*: Each universal function takes array inputs and produces array outputs by performing the core function element-wise on the inputs (where an element is generally a scalar, but can be a vector or higher-order sub-array for generalized ufuncs).

In [11]:
assert np.all([0,1,2,3] == np.arange(4))

In [12]:
assert np.all([1,1,1,1] == np.ones((1, 4)))

In [13]:
assert np.all([-1,0,1,2] == np.arange(4) - np.ones((1, 4)))

### Array creation

There are six general mechanisms for [creating arrays](https://numpy.org/doc/stable/user/basics.creation.html):

1. conversion from other Python structures (i.e., lists and tuples)
   - NumPy arrays can be defined using Python sequence such as lists `[...]` and tuples `(...)`
   - use `numpy.array` function to define a new array
   - the `dtype` of elements in the array can be specified explicitly
3. intrinsic NumPy array creation functions (e.g., arange, ones, zeros, etc.)
4. replicating, joining, or mutating existing arrays
5. reading arrays from disk, either from standard or custom formats
   - this is the most common case of large array creation
   - the [details](https://numpy.org/doc/stable/user/how-to-io.html#how-to-io) depend greatly on the format of data on disk
6. creating arrays from raw bytes through the use of strings or buffers
   - if the file has a relatively simple format then one can write a simple I/O library and use the NumPy fromfile() function and .tofile() method to read and write NumPy arrays directly (mind your byteorder though!)
8. use of special library functions (e.g., random)
   - many Python libraries, including SciPy, Pandas, and OpenCV, use NumPy ndarrays as the common format for data exchange

#### 1D array creation

These functions can be split into roughly three categories

- 1D array creation
- 2D array creation
- *n*-dimension array creation

#### 1D array creation

The 1D array creation functions e.g. `numpy.linspace` and `numpy.arange` generally need at least two inputs, `start` and `stop`.

`numpy.arange` creates arrays with regularly incrementing values, starting from `start` until `end` exclusive:

`numpy.linspace` will create arrays with a specified number of elements, and spaced equally between the specified beginning and end values inclusive.

In [14]:
assert np.all([ 0., 16., 32., 48., 64.] == np.linspace(0, 64, 5))

#### 2D array creation

The 2D array creation functions e.g. `numpy.eye`, `numpy.diag`, and `numpy.vander` define properties of special matrices represented as 2D arrays.

`np.eye(n)` defines a 2D identity matrix. The elements where $i=j$ (row index and column index are equal) are 1 and the rest are 0, also known as the *identity matrix*.

In [15]:
assert np.all(np.eye(3) == [
    [1,0,0],
    [0,1,0],
    [0,0,1]
])

`numpy.diag` can define either a square 2D array with given values along the diagonal and the rest are 0, also known as the *diagonal matrix*.

In [16]:
assert np.all(np.diag([1,2,3]) == [
    [1,0,0],
    [0,2,0],
    [0,0,3]
])

#### *n*-dimension array creation

The ndarray creation functions e.g. `numpy.ones`, `numpy.zeros`, and `random` define arrays based upon the desired shape. The ndarray creation functions can create arrays with any dimension by specifying how many dimensions and length along that dimension in a tuple or list.

`numpy.zeros` will create an array filled with 0 values with the specified shape.

In [17]:
assert np.all(np.zeros((2,3,4)) == [
    [[0.,0.,0.,0.],[0.,0.,0.,0.],[0.,0.,0.,0.]],
    [[0.,0.,0.,0.],[0.,0.,0.,0.],[0.,0.,0.,0.]]
])   # the default type is float64

`numpy.ones` will create an array filled with 1 values. It is identical to `numpy.zeros` in all other respects.

The `random` method of the result of `default_rng` will create an array filled with random values between 0 and 1. It is included with the `numpy.random` library. Below, two arrays are created with shapes (2,3) and (2,3,2), respectively. The seed is set to 42 so you can reproduce these pseudorandom numbers:

```python
from numpy.random import default_rng
arr1 = default_rng(42).random((2,3))      # 2D array of shape (2,3)
arr2 = default_rng(42).random((2,3,4))    # 3D array of shape (2,3,4)
```

### Higher dimensional arrays

Here, we can create a two-dimensional array by providing a list of lists, where each member of the inner list is a number.

In [18]:
vec = np.array([1,2])
assert np.all(vec.shape == (2,))

In [19]:
mat = np.array([[1,2],[3,4]])
assert np.all(mat.shape == (2,2))

Recall that under the hood, a NumPy array of any shape is a buffer containing the raw data as a flat (one-dimensional) array, and a collection of additional metadata that specifies details. One of the metadata is the `shape` that specifies the dimension of the NumPy array.

An array can be reshaped with little cost by simply changing the associated metadata. This is done using the `numpy.ndarray.reshape` method on the NumPy array:

In [20]:
mat2 = mat.reshape((4,))
# mat.reshape((4,))
assert np.all(mat2.shape == (4,))
assert np.all(mat2 == [1,2,3,4])
assert np.all(mat.shape == (2,2))
assert np.all(mat == [[1,2],[3,4]])

Note that the total number of elements must remain unchanged. The matrix `mat` originally has shape (2, 2) with a total of 4 elements, and the latter is a one-dimensional array with shape (4,), which again has a total of 4 elements. Attempting to reshape when there is a mismatch in the total number of elements will result in a `ValueError`.

Note that the `numpy.ndarray.reshape` method returns either a new view view object or a copy, depending on [the `order` and `copy` arguments](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html). In the above code, `mat2` is a reshaped view of `mat`: therefore

In [21]:
mat[0,1] = 10
assert mat2[1] == 10

## Matrices

NumPy arrays also serve as matrices, which are fundamental in mathematics and computational programming. A matrix is simply a two-dimensional array.

A matrix with $m$ rows and $n$ columns is usually described as an $m \times n$ matrix. A matrix that has the same number of rows as columns is said to be a *square matrix*.

In [22]:
A = np.arange(15).reshape(3,5)
assert np.all(A == [
    [ 0, 1, 2, 3, 4],
    [ 5, 6, 7, 8, 9],
    [10,11,12,13,14]
])
assert A.ndim == 2                  # two-dimensional
assert np.all(A.shape == (3,5))     # 3 rows, 5 columns

The *identity matrix* (of size $n$) is the $n \times n$ matrix where the $(i,i)$-th entry is 1, and the $(i,j)$-th entry is zero for $i \neq j$.

In [23]:
I3 = np.eye(3)
assert np.all(I3 == [
    [1,0,0],
    [0,1,0],
    [0,0,1]
])

### Basic methods

The *transpose* of a matrix is a matrix where rows and columns are interchanged.

- the `numpy.ndarray.transpose` method
- the `T` property of the `numpy.ndarray` object

In [24]:
a = A.transpose()
assert np.all(a.shape == (5,3))    # 5 rows, 3 columns
assert np.all(a == [
    [0,5,10],
    [1,6,11],
    [2,7,12],
    [3,8,13],
    [4,9,14]
])
assert np.all(a == A.T)

The *trace* of a matrix is the sum of the diagonal elements.

$$
\begin{aligned}
\text{trace}(B) &= \sum_{i=1}^{n} b_{i,i} \\
&= 0 + 4 + 8 = 12
\end{aligned}
$$

In [25]:
B = np.arange(9).reshape(3,3)
assert np.all(B == [
    [0,1,2],
    [3,4,5],
    [6,7,8]
])
assert B.trace() == 12

### Matrix multiplication

Formally, if $A$ is an $l \times m$ matrix, and $B$ is an $m \times n$ matrix, say

$$
A_{l,m} = \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,m} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,m} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{l,1} & a_{l,2} & \cdots & a_{l,m}
\end{pmatrix}
\text{ and }
B_{m,n} = \begin{pmatrix}
  b_{1,1} & b_{1,2} & \cdots & b_{1,n} \\
  b_{2,1} & b_{2,2} & \cdots & b_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  b_{m,1} & b_{m,2} & \cdots & b_{m,n}
\end{pmatrix}
$$

then the matrix product $C$ of $A$ and $B$ is an $l \times n$ matrix whose $(p,q)$-th entry is given by

$$
C_{p,q} = \sum_{i=1}^{m}a_{p,i}b_{i,q}
$$

Note that the number of columns of the first matrix must match the number of rows of the second matrix in order for matrix multiplication to be defined.

Python has an operator reserved for matrix multiplication `@`:

In [26]:
A = np.array([[ 1, 2],[ 3, 4]])
B = np.array([[-1, 1],[ 0, 1]])

In [27]:
assert np.all( A @ B == [
    [-1, 3],
    [-3, 7]
])

Matrix multiplication is not *commutative*.

In [28]:
assert np.all( B @ A == [
    [ 2, 2],
    [ 3, 4]
])

Matrix multiplication is fundamentally different from the component-wise multiplication of array `*`:

In [29]:
assert np.all( A * B == [
    [-1, 2],
    [ 0, 4]
])
assert np.all( B * A == A * B )

The *identity matrix* of size $n$ is the $n \times n$ square matrix with ones on the main diagonal and zeros elsewhere. When $A$ is an $m \times n$ matrix, it is a property of matrix multiplication that

$$
I_{m} A = A I_{n} = A
$$

In [30]:
A = np.arange(6).reshape(2,3)
I2 = np.eye(2)
assert np.all( A == I2 @ A )
I3 = np.eye(3)
assert np.all( A == A @ I3 )

### Determinants and inverses

A square matrix that has a non-zero determinant has a unique inverse, which translates to unique solutions of certain systems of equations.

For a $2 \times 2$ matrix

$$
A = \begin{pmatrix}
  a_{11} & a_{12} \\
  a_{21} & a_{22}
\end{pmatrix}
$$

the determinant of $A$ is defined by the formula

$$
\text{det}(A) = \begin{vmatrix}
  a_{11} & a_{12} \\
  a_{21} & a_{22}
\end{vmatrix}
= a_{11} a_{22} - a_{12} a_{21}
$$

For a $3 \times 3$ matrix

$$
A = \begin{pmatrix}
  a_{11} & a_{12} & a_{13} \\
  a_{21} & a_{22} & a_{23} \\
  a_{31} & a_{32} & a_{33}
\end{pmatrix}
$$

the determinant of $A$ is

$$
\text{det}(A)
= a_{11} a_{22} a_{33} - a_{11} a_{23} a_{32}
+ a_{12} a_{23} a_{31} - a_{12} a_{21} a_{33}
+ a_{13} a_{21} a_{32} - a_{13} a_{22} a_{31}
$$

For a general $n$-dimensional square matrix, refer to [Determinant at Wolfram](https://mathworld.wolfram.com/Determinant.html) and [Finding the determinant at SciPy](https://docs.scipy.org/doc/scipy/tutorial/linalg.html#finding-the-determinant).

The NumPy routine for computing the determinant of a matrix is `det` in the package `np.linalg`, which contains many routines for *linear algebra*.

> The `np.linalg.det` can carry a floating-point rounding error.

The SciPy also offers a `linalg` module that extends NumPy's `linalg`. The SciPy's `det` may be preferable if speed is important.

- [SciPy's linalg](https://docs.scipy.org/doc/scipy/tutorial/linalg.html) contains all the functions in `numpy.linalg`. plus some other more advanced ones not contained in `numpy.linalg`.
- Another advantage of using `scipy.linalg` over `numpy.linalg` is that it is always compiled with BLAS/LAPACK support, while for NumPy this is optional. Therefore, the SciPy version might be faster depending on how NumPy was installed.
- Therefore, unless you don’t want to add `scipy` as a dependency to your `numpy` program, use `scipy.linalg` instead of `numpy.linalg`.

In [31]:
import numpy as np
import scipy as sp
A = np.array([[ 1, 2],[ 3, 4]])
assert sp.linalg.det(A) == -2.   # det(A) = 1*4 - 2*3 = -2

The *inverse* of an $n \times n$ matrix $A$ is the (necessarily unique) $n \times n$ matrix $B$, such that $A B = B A = I$, where $I$ denotes the $n \times n$ identity matrix. When $A$ has an inverse, it is customary to denote it by $A^{-1}$.

$$
A A^{-1} = A^{-1} A = I
$$

> There will be a floating-point error in these computation due to the way that matrix inverses are computed.

In [32]:
Ainv = sp.linalg.inv(A)
assert np.all(np.isclose(Ainv, [[-2.,1.],[1.5,-0.5]]))
assert np.any(Ainv != [[-2.,1.],[1.5,-0.5]])   # due to a floating-point error

In [33]:
I = np.eye(2)
assert np.all(np.isclose(A @ Ainv, I))
assert np.any(A @ Ainv != I)    # due to a floating-point error
assert np.all(np.isclose(Ainv @ A, I))
assert np.any(Ainv @ A != I)    # due to a floating-point error

A [matrix function](https://docs.scipy.org/doc/scipy/tutorial/linalg.html#matrix-functions) can be defined using Taylor series expansion for the square matrix $A$ as

$$
f(A) = \sum_{k=0}^{\infty} \frac{f^{(k)}(0)}{k!} A^k
$$

`scipy.linalg` contains matrix functions such as

| method | description |
|--------|-------------|
| `expm` | the exponential function for matrices         |
| `logm` | the logarithm function for matrices           |
| `sinm` | the trigonometric function `sin` for matrices |
| `cosm` | the trigonometric function `cos` for matrices |
| `tanm` | the trigonometric function `tan` for matrices |

> Matrix functions are not the same as the standard `exp`. `log`, `sin`, `cos`, and `tan` functions in the base NumPy package, which apply the corresponding function on an element by element basis.

### Systems of equations

A system of linear equations can be written as

$$
\begin{equation}
    \begin{array}{c}
    a_{1,1} x_1 & + & a_{1,2} x_2 & + & \cdots & + & a_{1,n} x_n & = & b_1 \\
    a_{2,1} x_1 & + & a_{2,2} x_2 & + & \cdots & + & a_{2,n} x_n & = & b_2 \\
                &   &             &   &        &   &           & \vdots & \\
    a_{n,1} x_1 & + & a_{n,2} x_2 & + & \cdots & + & a_{n,n} x_n & = & b_n
    \end{array}
\end{equation}
$$

where
- $1 \lt n$
- $a_{i,j}$ and $b_i$ are known values

and the $x_i$ values are the unknown values that we wish to find.

Let
- $A$ be the matrix containing the coefficients taken from the system of equations,
- $x$ be the column vector containing $x_i$ values, and
- $b$ be the column vector containing $b_i$ values

as

$$
A = \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{n,1} & a_{n,2} & \cdots & a_{n,n} 
\end{pmatrix}
\;\text{,}\;
x = \begin{pmatrix}
  x_1 \\ x_2 \\ \vdots \\ x_n
\end{pmatrix}
\;\text{,}\;
b = \begin{pmatrix}
  b_1 \\ b_2 \\ \vdots \\ b_n
\end{pmatrix}
$$

then we can rewrite the system of equations as the single matrix equation

$$
A x = b
$$

To solve this matrix equation, we use the `solve` routine in the `linalg` module. For example, provided the following system of equation:

$$
\begin{array}{r}
  3 x_1 & - & 2 x_2 & + &   x_3 & = &  7 \\
    x_1 & + &   x_2 & - & 2 x_3 & = & -4 \\
 -3 x_1 & - & 2 x_2 & + &   x_3 & = &  1
\end{array}
$$

In [34]:
A = np.array([[3,-2,1],[1,1,-2],[-3,-2,1]])
b = np.array([7,-4,1])
x = sp.linalg.solve(A,b)
assert np.all(np.isclose(A @ x, b))
# assert np.all((A @ x) != b)
assert np.all(np.isclose(x, [1,-1,2]))
assert np.all(x != [1,-1,2])

Recall that

$$
\text{the system has a unique solution} \iff
\text{the determinant of this matrix A is not zero}
$$

If the determinant is zero, one of two things happens:

- the system can have no solution (the system is *inconsistent*)
- the system can infinitely many solutions (the system is *under-specified*)

For example, the following system is inconsistent:

$$
\begin{array}{r}
    x & + &   y & = & 1 \\
  2 x & + & 2 y & = & 1
\end{array}
$$

The following system is consistent but under-specified, therefore have no unique solution:

$$
\begin{array}{r}
    x & + &   y & = & 1 \\
  2 x & + & 2 y & = & 2
\end{array}
$$

The coefficient matrix does not need to be square for the system to be solvable. For example, if there are more equations than there are unknown values (a coefficient matrix has more rows than columns). Such a system is said to be *over-specified* and, provided that it is consistent, it will have a solution.

If there are fewer equations than there are unknown values, then the system is said to be under-specified. Under-specified systems of equations generally have infinitely many solutions if they are consistent, since there is not enough information to uniquely specify all the unknown values.

### Eigenvalues and eigenvectors

Consider the matrix equation $A \mathbf{x} = \lambda \mathbf{x}$, where

- $A$ is a square $(n \times n)$ matrix
- $\mathbf{x}$ is a vector $\neq \{ \mathbf{0} \}$
- $\lambda$ is a number

If there is an $\lambda$ and an $\mathbf{x}$ that solves this equation, then

- $\lambda$ is called the *eignevalue* (/ˈaɪɡən-/)
- $\mathbf{x}$ is called the *eigenvector*

It is perfectly possible for a matrix with only real entries to have complex eigenvalues and eigenvectors.

- $\lambda \in \mathbb{C}$
- $\mathbf{x} \in \mathbb{C}^n \setminus \{ \mathbf{0} \}$

For example, take the following matrix

$$
A = \begin{pmatrix}
   3 & -1 &  4 \\
  -1 &  0 & -1 \\
   4 & -1 &  2
\end{pmatrix}
$$

Use the `eig` routine in the `linalg` modules to find the eigenvalues and eigenvectos of a square matrix.

In [35]:
A = np.array([
    [ 3,-1, 4],
    [-1, 0,-1],
    [ 4,-1, 2]
])
v, B = sp.linalg.eig(A)

In [36]:
v

array([ 6.82315616+0.j, -1.53711415+0.j, -0.28604201+0.j])

In [37]:
B

array([[ 0.73271846, -0.65131465,  0.19726352],
       [-0.20260301,  0.06794858,  0.97690072],
       [ 0.64967352,  0.75575937,  0.08217113]])

The `eig` routine returns a pair $(v, B)$ where

- $v$ is a one-dimensional array containing the eigenvalues
- $B$ is a two-dimensional array whose columns are the corresponding eigenvectors

We can extract the eigenvector for an eigenvalue with the form of `B[:i]`:

In [38]:
v[0]  # the first eigenvalue

np.complex128(6.823156164525969+0j)

In [39]:
B[:,0]   # the eigenvector that corresponds the first eigenvalue

array([ 0.73271846, -0.20260301,  0.64967352])

One key application of eigenvalues and eigenvectors is in *principle component analysis*, which is a key technique for reducing a large, complex dataset to better understand the internal structure.

We can only compute eigenvalues and eigenvectors for square matrices; for non-square matrices, the definition does not make sense. There is a generalization of eigenvalue, and eigenvalues to non-square matrices is called *singular values*.

#### Norm

We can check that these values do indeed satisfy the definition of an eigenvalue/eigenvector pair as follows:

In [40]:
l0 = v[0]
x0 = B[:,0]
assert np.isclose(0, sp.linalg.norm( A @ x0 - l0 * x0 ))
assert 0 != sp.linalg.norm( A @ x0 - l0 * x0 )

The `norm` computed here represent the "distance" between $A \mathbf{x}$ and $\lambda \mathbf{x}$. The fact that this is not zero is likely due to floating-point precision error.

- The [norm](https://mathworld.wolfram.com/Norm.html) of a mathematical object is a quantity that in some (possibly abstract) sense describes the length, size, or extent of the object.
- Norms exist for several kinds of mathematical objects, such as complex numbers (complex modulus),  vectors (vector norms), and matrices (matrix norms).

For example, given an $n$-dimensional vector $\mathbf{x}$, a general [vector norm](https://mathworld.wolfram.com/VectorNorm.html) $| \mathbf{x} |$, is a non-negative norm defined such that:
- $| \mathbf{x} | \geq 0$
- $| \mathbf{x} | = 0 \iff \mathbf{x} = \mathbf{0}$
- $| k \: \mathbf{x} | = | k | \: | \mathbf{x} |$ for any scalar $k$
- $| \mathbf{x} + \mathbf{y} | \leq | \mathbf{x} | + | \mathbf{y} |$

The most commonly encountered vector norm (often simply called "the norm" of a vector, or sometimes the magnitude of a vector) is the L2-norm, given by

$$
| x |_2 = | x | = \sqrt{x_1^2 + x_2^2 + \cdots + x_n^2}
$$

### Sparse matrices

A matrix is a *sparse matrix* if a large number of the elements are zero.

The `scipy.sparse` [module](https://docs.scipy.org/doc/scipy/tutorial/sparse.html) contains the following formats, each with their own distinct advantages and disadvantages:

- Block Sparse Row (BSR) arrays `scipy.sparse.bsr_array`, which are most appropriate when the parts of the array with data occur in contiguous blocks.
- Coordinate (COO) arrays `scipy.sparse.coo_array`, which provide a simple way to construct sparse arrays and modify them in place. COO can also be quickly converted into other formats, such CSR, CSC, or BSR.
- Compressed Sparse Row (CSR) arrays `scipy.sparse.csr_array`, which are most useful for fast arithmetic, vector products, and slicing by row.
- Compressed Sparse Column (CSC) arrays `scipy.sparse.csc_array`, which are most useful for fast arithmetic, vector products, and slicing by column.
- Diagonal (DIA) arrays `scipy.sparse.dia_array`, which are useful for efficient storage and fast arithmetic so long as the data primarily occurs along diagonals of the array.
- Dictionary of Keys (DOK) arrays `scipy.sparse.dok_array`, which are useful for fast construction and single-element access.
- List of Lists (LIL) arrays `scipy.sparse.lil_array`, which are useful for fast construction and modification of sparse arrays.

All formats of `scipy.sparse` arrays can be constructed directly from a `numpy.ndarray`.

In [41]:
dense = np.array([
    [1, 0, 0, 2],
    [0, 4, 1, 0],
    [0, 0, 5, 0]
])
coo = sp.sparse.coo_array(dense)
assert np.all(dense == coo.todense())

However, some sparse formats can be constructed in different ways, too. Each sparse array format has different strengths, and these strengths are documented in each class. For example, one of the most common methods for constructing sparse arrays is to build a sparse array from the individual row, column, and data values. For our array from before:

In [42]:
row = [ 0, 0, 1, 1, 2]
col = [ 0, 3, 1, 2, 2]
dat = [ 1, 2, 4, 1, 5]
csr = sp.sparse.csr_array((dat, (row, col)))
assert np.all(dense == csr.todense())

The `scipy.linalg.sparse` modules also contains many of the routines that can be found in the `linalg` module of NumPy (or SciPy) that accept sparse matrices instead of full NumPy arrays, such as `eig` and `inv`. 

## Summary

Python offers built-in support for mathematics with some basic numeral types, arithmetic, and basic mathematical functions. However, for more serious computations involving large arrays of numerical values, you should use the Numpy and SciPy packages.

- NumPy provides high-performance array types and basic routines
- SciPy provides more specific tools for solving equations and working with sparse matrices

[SciPy](https://docs.scipy.org/doc/scipy-1.15.0/tutorial/index.html) is a collection of mathematical algorithms and convenience functions built on NumPy. The [NumPy linear algebrahttps://numpy.org/doc/stable/reference/routines.linalg.html]() functions rely on BLAS and LAPACK to provide efficient low level implementations of standard linear algebra algorithms.

- The [BLAS (Basic Linear Algebra Subprograms)](https://www.netlib.org/blas/) are routines that provide standard building blocks for performing basic vector and matrix operations.
  - BLAS originated as a Fortran library in 1979 and its interface was standardized by the BLAS Technical Forum. ([wikipedia](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms))
- The [LAPACK (Linear Algebra PACKage)](https://www.netlib.org/lapack/) is written in Fortran 90 and provides routines for solving systems of simultaneous linear equations, least-squares solutions of linear systems of equations, eigenvalue problems, and singular value problems.
  - LAPACK routines are written so that as much as possible of the computation is performed by calls to the Basic Linear Algebra Subprograms (BLAS). LAPACK is designed at the outset to exploit the Level 3 BLAS &mdash; a set of specifications for Fortran subprograms that do various types of matrix multiplication and the solution of triangular systems with multiple right-hand sides.
  - LAPACK was initially released in 1992. ([wikipedia](https://en.wikipedia.org/wiki/LAPACK))